# LSTM model 1 on KaggleCompetition data set 

## Importing packages 

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import os.path as op
import time
import matplotlib.pyplot as plt
#Loading tensorflow and keras 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from keras.utils import plot_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,GlobalMaxPool1D,Dropout,concatenate
#Loading nltk
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#Loading sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
#Loading K-folds
from sklearn.model_selection import StratifiedKFold
#Split data 
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


Formula 3 =  ( correct_claims_per_ src / correct_claims_total ) - ( incorrect_claims_per_ src / incorrect_claims_total )

## Paths 

In [2]:
DATA_DIR = '../../../../../Data'

## Loading Data 

In [3]:
# Load train data
train = pd.read_csv(op.join(DATA_DIR, 'Kaggle_competition/Claims_to_work_with/claims.csv'))
train

id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiumnews.com   
3                                Jessica Purkiss   
4                                 Howard Portnoy   
...                                          ...   
20795                              Jerome Hudson   
20796                           Benjamin Hoffman   
20797  Michael J. de la Merced and Rachel Abrams   
20798                                Alex Ansary   
20799                              David Swanson   

                                                    text  label  
0      House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1      Ever get the feeling your life circles the rou...      0  
2      Why the Truth Might Get You Fired October 29, ...      1  
3      Videos 15 Civilians Killed In Single US Airstr...      1  
4      Print \nAn Iranian woman has been sentenced to...      1  
...                                                  ...    ...  
20795  Rapper T. I. unloaded on black celebrities who...      0  
20796  When the Green Bay Packers lost to the Washing...      0  
20797  The Macy’s of today grew from the union of sev...      0  
20798  NATO, Russia To Hold Parallel Exercises In Bal...      1  
20799    David Swanson is an author, activist, journa...      1  

[20800 rows x 5 columns]

In [4]:
train['author'].value_counts()

Pam Key                      243
admin                        193
Jerome Hudson                166
Charlie Spiering             141
John Hayward                 140
                            ... 
Hari Kumar and Nida Najar      1
Alice Salles                   1
Amanda Monteiro                1
AZU                            1
Vladimir Putin                 1
Name: author, Length: 4201, dtype: int64

In [5]:
# Find Na 
train.isnull().sum()

id           0
title        0
author    1957
text         0
label        0
dtype: int64

In [6]:
train = train[train['author'].notna()]

In [7]:
train=train.reset_index(drop=True)

In [8]:
#Get the Depndent feature
data_features=train.drop('label',axis=1)
data_label=train['label']
# set vocabulary size
vo_size=500
random_state_value=42
sent_length = 22
padding_form='pre'
embedding_vector_feature = 10
result_list=[]
size_comb=[0.05,0.1, 0.15, 0.2, 0.25, 0.3]

In [9]:
#Dataset Preprocessing
def data_Preprocessing(data): 
    ps =PorterStemmer()
    corpus = []
    for i in range(0, len(data)):
        print("Status: %s / %s" %(i, len(data)), end="\r")
        review = re.sub('[^a-zA-Z]', ' ',data['title'][i])
        review = review.lower()
        review = review.split()
    
        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
    
    return corpus

In [10]:
def source_credibility_formule3(data_sources):
    #True claims told by sources  
    values_0=(train.loc[train['label'] == 0, 'author'].value_counts(dropna=True).keys().tolist())
    counts_0=(train.loc[train['label'] == 0, 'author'].value_counts(dropna=True).tolist())
    claims_0 = dict(zip(values_0, counts_0))
    #False claims told by sources 
    values_1=(train.loc[train['label'] == 1, 'author'].value_counts(dropna=True).keys().tolist())
    counts_1=(train.loc[train['label'] == 1, 'author'].value_counts(dropna=True).tolist())
    claims_1 = dict(zip(values_1, counts_1))
    
    #Calculate the " correct_claims_src/correct_claims_total "
    score_sources0={}
    
    all_values = claims_0.values()
    total_correct_claims = sum(all_values)
    
    for k1,v1 in claims_0.items():
            term_0=v1/total_correct_claims
            score_src=term_0
            score_sources0.update( {k1 :score_src } )
                
    #Calculate the " incorrect_claims_src/ incorrect_claims_total "          
    score_sources1={}
    
    all_values = claims_1.values()
    total_incorrect_claims = sum(all_values)
    
    for k2,v2 in claims_1.items():
            term_1=v2/total_incorrect_claims
            score=term_1
            score_src=score
            score_sources1.update( {k2 :score_src } )
                
    #Calculate the final term "term_0 - term_1"
    score_sources={}
    for k3, v3 in score_sources0.items(): 
        for k4,v4 in score_sources1.items():
            if k3==k4 :
                score_src=(v3-v4)
                score_sources.update({k3 :score_src } )
            if k3 not in score_sources1.keys():
                score_src= v3
                score_sources.update( {k3 :score_src } )  
                
    for k4,v4 in score_sources1.items():
        if k4 not in score_sources0.keys():
            score_src= v4 * (-1)
            score_sources.update( {k4 :score_src } )
                
   
   #Normalise the data 
    score_sources_final={}
    
    all_values = score_sources.values()
    maxval = max(all_values)
    minval= min(all_values)
    
    for k,v in score_sources.items():
        z=(v-minval)/(maxval-minval)
        z=z*100
        score_sources_final.update({k : z } ) 
        
    #Transform it to a list 
    val=score_sources_final.values()
    list_cred_src=list(val)  
    
    
    return list_cred_src,score_sources_final

In [11]:
def embedding (onehot_rep,sent_length,padding_form,list_cred):
    embedded_doc=pad_sequences(onehot_rep, padding=padding_form, maxlen=sent_length)
    z=0
    r=0
    while (z < len(embedded_doc)) & (r < len(list_cred)) :
        embedded_doc[z][0]=list_cred[r]
        z=z+1
        r=r+1  
    return embedded_doc

In [12]:
def source_credibility_formule3_test(data_sources_test,score_sources_final):
    data_sources_test=data_sources_test.reset_index(drop=True)
    score_sources_test={}
    for t in range(len(data_sources_test)):
        for k,v in score_sources_final.items():
            if k == data_sources_test['author'][t]:
                score_sources_test.update({k:v})   
    val2=score_sources_test.values()
    list_cred_src_test=list(val2)
    return list_cred_src_test,score_sources_test


In [13]:
def Train_model(test_size,vo_size,data_features, data_label):
    x_train, x_test, y_train, y_test = train_test_split(data_features,data_label,test_size=test_size_value,random_state=random_state_value)  
    #Reset the index 
    x_train=x_train.reset_index(drop=True)
    x_test=x_test.reset_index(drop=True)
    y_train=y_train.reset_index(drop=True)
    y_test=y_test.reset_index(drop=True)
    
    authors=x_test['author'].value_counts()
    #Preprocessing of the training dataset 
    data=x_train
    corpus_train= data_Preprocessing(data)
    #Calculate the source credibility 
    data_sources=train
    list_cred_src,score_sources_final= source_credibility_formule3(data_sources)
    #one hot representation
    onehot_rep_train = [one_hot(words, vo_size) for words in corpus_train]
    # pad_sequences
    embedded_doc_train= embedding (onehot_rep_train,sent_length,padding_form,list_cred_src)
    
    #Preprocessing of the testing dataset 
    data_test=x_test
    corpus_test= data_Preprocessing(data_test)
    #one hot representation
    onehot_rep_test = [one_hot(words, vo_size) for words in corpus_test]
    #Calculate the source credibility for the testing dataset 
    score_sources_final=score_sources_final
    data_sources_test=x_test
    list_cred_src_test,score_sources_test=source_credibility_formule3_test(data_sources_test,score_sources_final)
    # pad_sequences for the testing dataset 
    embedded_doc_test= embedding (onehot_rep_test,sent_length,padding_form,list_cred_src_test)
    
    #Prepare the data for the model
    x_final_train=np.array(embedded_doc_train)
    y_final_train=np.array(y_train)
    
    x_final_test=np.array(embedded_doc_test)
    y_final_test=np.array(y_test)
   
    #the model 1
    model = Sequential()
    model.add(Embedding(vo_size,embedding_vector_feature,input_length=sent_length))
    model.add(LSTM(100))
    model.add(Dense(1,activation='sigmoid')) 
    
    return model, x_final_train, y_final_train, x_final_test, y_final_test, list_cred_src_test,list_cred_src,authors

In [14]:

#Main program 
""""total_len=0
total_len2=0
len_authors_total_fianl=0
len_authors_total=0
total_results_final=[]
total_len1_final=  0
total_len2_final=  0
l=0
total_results=[]"""
for comb in (size_comb):
    len1=0
    len2=0
    e=0
    len_authors=0
    test_size_value=comb
    model, x_final_train, y_final_train, x_final_test, y_final_test, list_cred_src_test,list_cred_src,authors=Train_model(test_size_value,vo_size,data_features, data_label)
    len_authors=len(authors)
    x = [e for e in list_cred_src_test if e > 50]
    len1=len(x)
    #total_len=total_len+len1
    x1 = [e1 for e1 in list_cred_src_test if e1 == 0]
    len2=len(x1)
    #total_len2=total_len2+len2
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model 
    history=model.fit(x_final_train,y_final_train,validation_split=0.1,epochs=10,batch_size=64)
    # Evaluate the model 
    results = model.evaluate(x_final_test, y_final_test, batch_size=64)
    """"total_results[0]=results[l]+total_results[0]
    total_results[1]=results[l+1]+total_results[1]
    l=l+1
    len_authors_total=len_authors+len_authors_total"""
        
    
    # Stock the results 
    """"len_authors_total_fianl=len_authors_total/10
    total_results_final[0]=total_results[0]/10
    total_results_final[1]=total_results[1]/10
    total_len1_final=  total_len1/10
    total_len2_final=  total_len2/10"""
    item=(comb, y_final_train.shape, y_final_test.shape,len1,len2,results,authors)
    result_list.append(item)
    #Save the figures of accuracy and loss  
    '''comb1=str(comb)
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.savefig('formule_1_accuracy '+comb1+'.png')
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.savefig('formule_1_loss '+comb1+'.png')'''

Train on 16110 samples, validate on 1790 samples
Epoch 1/10
16110/16110 [==============================] - 9s 545us/sample - loss: 0.3951 - accuracy: 0.8094 - val_loss: 0.2490 - val_accuracy: 0.8994
Epoch 2/10
16110/16110 [==============================] - 6s 363us/sample - loss: 0.2182 - accuracy: 0.9080 - val_loss: 0.2429 - val_accuracy: 0.8911
Epoch 3/10
16110/16110 [==============================] - 6s 362us/sample - loss: 0.1987 - accuracy: 0.9178 - val_loss: 0.2280 - val_accuracy: 0.8983
Epoch 4/10
16110/16110 [==============================] - 6s 365us/sample - loss: 0.1889 - accuracy: 0.9217 - val_loss: 0.2236 - val_accuracy: 0.9056
Epoch 5/10
16110/16110 [==============================] - 6s 385us/sample - loss: 0.1805 - accuracy: 0.9247 - val_loss: 0.2279 - val_accuracy: 0.8955
Epoch 6/10
16110/16110 [==============================] - 6s 403us/sample - loss: 0.1754 - accuracy: 0.9272 - val_loss: 0.2316 - val_accuracy: 0.9061
Epoch 7/10
16110/16110 [===========================

4711/4711 [==============================] - 1s 150us/sample - loss: 0.2336 - accuracy: 0.9030
Train on 11871 samples, validate on 1319 samples
Epoch 1/10
11871/11871 [==============================] - 13s 1ms/sample - loss: 0.4450 - accuracy: 0.7760 - val_loss: 0.2730 - val_accuracy: 0.8923
Epoch 2/10
11871/11871 [==============================] - 7s 609us/sample - loss: 0.2321 - accuracy: 0.9047 - val_loss: 0.2386 - val_accuracy: 0.8939
Epoch 3/10
11871/11871 [==============================] - 7s 618us/sample - loss: 0.2114 - accuracy: 0.9118 - val_loss: 0.2420 - val_accuracy: 0.8908
Epoch 4/10
11871/11871 [==============================] - 8s 656us/sample - loss: 0.2024 - accuracy: 0.9156 - val_loss: 0.2288 - val_accuracy: 0.9037
Epoch 5/10
11871/11871 [==============================] - 7s 565us/sample - loss: 0.1918 - accuracy: 0.9210 - val_loss: 0.2537 - val_accuracy: 0.8832
Epoch 6/10
11871/11871 [==============================] - 7s 554us/sample - loss: 0.1840 - accuracy: 0.9243

In [15]:
result_list

[(0.05,
  (17900,),
  (943,),
  217,
  1,
  [0.2219133799301099, 0.9109226],
  admin                                                                14
  Jerome Hudson                                                        13
  Pam Key                                                              11
  Activist Post                                                         9
  Ben Kew                                                               9
                                                                       ..
  Theresa Crouse                                                        1
  Gardiner Harris, John Schwartz, Jackie Calmes and Jonathan Martin     1
  Lance Schuttler                                                       1
  Pam Belluck                                                           1
  Alexander Burns and Jonathan Martin                                   1
  Name: author, Length: 589, dtype: int64),
 (0.1,
  (16958,),
  (1885,),
  343,
  1,
  [0.22670179530780574,